In [1]:
import numpy as np
import pandas as pd
import xgboost

df=pd.read_csv('train.csv')
df_test=pd.read_csv('test.csv')

In [2]:
X=df.iloc[:,:-1]
X_Sub=df_test.iloc[:,:]
y=df.iloc[:,-1]

In [3]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=27)

In [4]:
df_X = pd.concat([X_train, y_train], axis=1)

In [6]:
df_majority = df_X[df_X.Response==0]
df_minority = df_X[df_X.Response==1]

In [8]:
# Upsample minority class
from sklearn.utils import resample
df_minority_upsampled = resample(df_minority,replace=True,n_samples=len(df_majority),random_state=1234)
 


In [10]:
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
df_upsampled.Response.value_counts()

1    30985
0    30985
Name: Response, dtype: int64

In [11]:
X_train=df_upsampled.iloc[:,:-1]
y_train=df_upsampled.iloc[:,-1]

In [12]:
X_train=X_train.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]
X_test=X_test.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]
X_Sub=X_Sub.loc[:,['Reco_Policy_Premium','Region_Code','Reco_Policy_Cat','Upper_Age','Holding_Policy_Duration','Reco_Insurance_Type']]

In [13]:
cat=['Reco_Insurance_Type']
num=[i for i in X_train.columns if i not in cat]

In [14]:
X1_train=X_train.loc[:,cat]
X2_train=X_train.loc[:,num]

In [15]:
X1_test=X_test.loc[:,cat]
X2_test=X_test.loc[:,num]

X1_Sub=X_Sub.loc[:,cat]
X2_Sub=X_Sub.loc[:,num]

In [16]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
X1_train=X1_train.apply(le.fit_transform)

In [17]:
X1_test=X1_test.apply(LabelEncoder().fit_transform)
X1_Sub=X1_Sub.apply(LabelEncoder().fit_transform)

In [18]:
unique_items2=X2_train['Holding_Policy_Duration'].unique()
map_dict2={}
for i,j in enumerate(unique_items2):
    map_dict2[j]=i
map_dict2[np.nan]=np.nan
X2_train['Holding_Policy_Duration']=X2_train['Holding_Policy_Duration'].map(map_dict2)

In [19]:
X2_test['Holding_Policy_Duration']=X2_test['Holding_Policy_Duration'].map(map_dict2)
X2_Sub['Holding_Policy_Duration']=X2_Sub['Holding_Policy_Duration'].map(map_dict2)

In [20]:
X_train=pd.concat([X1_train,X2_train],axis=1)
X_test=pd.concat([X1_test,X2_test],axis=1)
X_Sub=pd.concat([X1_Sub,X2_Sub],axis=1)

In [21]:
clf = xgboost.XGBClassifier()
clf.fit(X_train,y_train)

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:23:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=8, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [22]:
y_pred=clf.predict(X_test)

In [23]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, y_pred)

0.5863640026522526

In [24]:
from sklearn.metrics import accuracy_score, confusion_matrix
accuracy_score(y_test, y_pred)

0.6063672988110446

In [25]:
confusion_matrix(y_test, y_pred)

array([[4809, 2879],
       [1127, 1362]], dtype=int64)